In [1]:
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np

def pearson_spearman(x, y):
    pearson_corr, pearson_p_value = scipy.stats.pearsonr(x, y)
    print(f"Pearson correlation coefficient: {pearson_corr:.3f}, p-value: {pearson_p_value:.3g}")

    spearman_corr, spearman_p_value = scipy.stats.spearmanr(x, y)
    print(f"Spearman correlation coefficient: {spearman_corr:.3f}, p-value: {spearman_p_value:.3g}")
    
def collapse_if_identical(lst):
    return lst[0] if all(x == lst[0] for x in lst) else lst
    

# takes in list and returns peak with largest signal
def keep_largest_signal(lst, qn_df):
    if isinstance(lst, list):
        maxPeakName = ''
        maxPeakStrength = 0
        for peak in lst:
            matching_rows = qn_df[qn_df[3] == peak]
            if not matching_rows.empty:
                signal = matching_rows[4].iloc[0]
                if signal > maxPeakStrength:
                    maxPeakStrength = signal
                    maxPeakName = peak
        return maxPeakName
    else: 
        return lst

# def overlap(start1, end1, start2, end2):
#     return max(0, min(end1, end2)-max(start1, start2))

def get_biggest_overlap(lst, col, df):
    maxOverlap = 0
    maxPeakOverlap = ""
    for peak in lst:
        row = df[df[col] == peak].iloc[0] #iloc bc possible to return multiple rows (never will based on th
        overlap = max(0, min(row[2], row[12])-max(row[1], row[11]))
        if overlap > maxOverlap:
            maxOverlap = overlap
            maxPeakOverlap = peak
    return peak

In [26]:
# EXAMPLE MAKE ONE_TO_ONE_PEAKS THEN SAVE IT

species = 'macaque'
both_df = pd.read_csv(f"/home/azstephe/liverRegression/regression_liver/data/splits/{species}Mouse/{species}ToMouse_liver_{species}Enhancer_mouseEnhancer_val_wawb.narrowPeak", sep="\t", header=None)
nonMouse_true = pd.read_csv(f"/home/azstephe/liverRegression/regression_liver/data/log/{species}_liver_pos_ALL.bed", sep="\t", header=None)
mouse_true = pd.read_csv("/home/azstephe/liverRegression/regression_liver/data/log/mouse_liver_pos_ALL.bed", sep="\t", header=None)

unique1_2 = both_df[~both_df.duplicated(subset=[1, 2], keep=False)] # entries with unique mac mapped to mouse start and end
all_unique = unique1_2[~unique1_2.duplicated(subset=[11,12], keep=False)]

#### COL1,2 DUPLICATES

duplicates3 = both_df[both_df.duplicated(subset=[1, 2], keep=False)] # entries with duplicated mac mapped to mouse start and end 
grouped_dups3 = duplicates3.groupby([1, 2])[3].apply(list).reset_index(name='col3')

# grouped dups: start | end | [peaks with these endpoints]
grouped_dups3['col3'] = grouped_dups3['col3'].apply(collapse_if_identical) 

grouped_dups3['col3'] = grouped_dups3['col3'].apply(lambda x: keep_largest_signal(x, nonMouse_true)) # redundant endpoints now map to 1 peak in col3
grouped_dups3 = grouped_dups3[grouped_dups3['col3'] != ''] # remove macaque peaks that aren't large enough

# df of duplicated col1,2 with strongest peak
keep_strongestcol3 = duplicates3.merge(grouped_dups3[['col3']], left_on=3, right_on='col3', how='inner').drop('col3', axis=1) # keeps the strongest signal in col3 for redundant endpoints

#### COL11,12 DUPLICATES

grouped_dups13 = keep_strongestcol3.groupby([11, 12])[13].apply(list).reset_index(name='col13')
grouped_dups13['col13'] = grouped_dups13['col13'].apply(collapse_if_identical)

grouped_dups13['col13'] = grouped_dups13['col13'].apply(lambda x: keep_largest_signal(x, mouse_true))
grouped_dups13 = grouped_dups13[grouped_dups13['col13'] != '']

keep_strongestcol13 = keep_strongestcol3.merge(grouped_dups13[['col13']], left_on=13, right_on='col13', how='inner').drop('col13', axis=1)

unique_endpoints = keep_strongestcol13 #rows with unique endpoints from the duplicated endpoints set

####

# col3 peakname duplicates with different endpoints
still_dups_col3 = unique_endpoints[unique_endpoints.duplicated(subset=[3], keep=False)]

# col3peaks | [col13 peaks intersecting col3 peak]
grouped_dcol3 = still_dups_col3.groupby(3)[13].apply(list).reset_index(name='col13')

# get the col13 peak with most overlap of col3
grouped_dcol3['col13'] = grouped_dcol3['col13'].apply(lambda x: get_biggest_overlap(x, 13, still_dups_col3))

merged3 = still_dups_col3.merge(grouped_dcol3[[3, 'col13']], left_on=[3, 13], right_on=[3, 'col13'], how='left', indicator=True)

remove3 = merged3[merged3['_merge'] == 'left_only'].drop(columns=['_merge']) # col13 is what we want to remove

unique_endpoints_subset = unique_endpoints.iloc[:,:20]
remove3_subset = remove3.iloc[:,:20]

# all col3 entries unique
unique3 = unique_endpoints[~unique_endpoints_subset.apply(tuple, axis=1).isin(remove3_subset.apply(tuple, axis=1))] 

####

# col13 peakname duplicates with different endpoints
still_dups_col13 = unique3[unique3.duplicated(subset=[13], keep=False)]

# col13peaks | [col3 peaks intersecting col13 peak]
grouped_dcol13 = still_dups_col13.groupby(13)[3].apply(list).reset_index(name='col3')

# get the col3 peak with most overlap of col13
grouped_dcol13['col3'] = grouped_dcol13['col3'].apply(lambda x: get_biggest_overlap(x, 3, still_dups_col13))

merged13 = still_dups_col13.merge(grouped_dcol13[[13, 'col3']], left_on=[3, 13], right_on=['col3', 13], how='left', indicator=True)

remove13 = merged13[merged13['_merge'] == 'left_only'].drop(columns=['_merge'])

unique3_subset = unique3.iloc[:, :20]
remove13_subset = remove13.iloc[:, :20]

# Identify rows in u3 that are NOT in remove3
filtered_peaks_unique = unique3[~unique3_subset.apply(tuple, axis=1).isin(remove13_subset.apply(tuple, axis=1))]

full_unique = pd.concat([all_unique, filtered_peaks_unique])

# Sort by column '1'
one_to_one_peaks = full_unique.sort_values(by=1).reset_index(drop=True)
one_to_one_peaks.to_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/oneToOnePeaks_val/{species}_mouse.bed', header=None, sep='\t')
one_to_one_peaks
##############################



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,chr9,6668262,6668828,peak24458,-1,.,-1,-1,-1,370,chr9,6668246,6669123,peak20464,1000,.,10.41801,106.98440,104.00175,372
1,chr9,7603389,7603594,peak64793,-1,.,-1,-1,-1,114,chr9,7603184,7603794,peak20478,1000,.,18.10567,247.44370,243.99432,315
2,chr9,7914492,7914988,peak729,-1,.,-1,-1,-1,141,chr9,7914173,7914971,peak20484,1000,.,15.21739,160.17427,156.98676,531
3,chr8,8733638,8734157,peak30058,-1,.,-1,-1,-1,379,chr8,8733751,8734148,peak19418,1000,.,12.36603,110.58308,107.58508,270
4,chr8,8992445,8993220,peak5261,-1,.,-1,-1,-1,506,chr8,8992509,8993074,peak19424,1000,.,6.54450,48.63282,45.97403,451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,chr8,126824543,126825031,peak57495,-1,.,-1,-1,-1,266,chr8,126824525,126825188,peak20431,1000,.,6.61796,38.65833,36.08059,314
256,chr8,126838305,126839235,peak50766,-1,.,-1,-1,-1,182,chr8,126839044,126839256,peak20435,829,.,5.00771,21.68738,19.29232,109
257,chr8,126849473,126850187,peak58047,-1,.,-1,-1,-1,560,chr8,126849637,126850309,peak20437,836,.,4.81874,20.20660,17.83258,402
258,chr8,126920866,126921347,peak14712,-1,.,-1,-1,-1,189,chr8,126920865,126921188,peak20441,1000,.,6.42499,33.74764,31.21361,172


In [46]:
species = 'macaque'
model = 'bdbi7l3n'

nonMouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/{species}_liver_pos_ALL.bed', sep="\t", header=None)
mouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/mouse_liver_pos_ALL.bed', sep="\t", header=None)

one_to_one_peaks = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/oneToOnePeaks/{species}_mouse.bed', header=None, sep='\t')
pred_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model}/activations_{species}_TEST.csv', header=None)
val1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/amy_test1/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
# val2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/val2/{species}_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
val3_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_test3/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]

val2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_test2/{species}_liver_TEST_500bp.bed', header=None, sep="\t").rename(columns={0: 'mac_chr'})

val1_len = 2*len(val1_df)
val2_len = 2*len(val2_df)

####################### make the ____ doubled predicted and true df
peaks = one_to_one_peaks[[4, 14]]

merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_hello'))

# Merge the result with df3 on col13 and df3['col3']
merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_MAC', '_mouse'))

# Subtract col4 values from df2 and df3
# merged_df['result'] = merged_df['4_mac'] - merged_df['4_mouse']
# res = merged_df[['4', 14, '4_mac', '4_mouse', 'result']]
# nan = res[res.isna().any(axis=1)]

# nan

####################### 

doubled_val2_df = pd.concat([val2_df, val2_df]).sort_index(kind='mergesort').reset_index(drop=True)

pred_val2_df = pred_df.iloc[val1_len:val1_len + val2_len].reset_index(drop=True)

doubled_combined = pd.concat([doubled_val2_df, pred_val2_df], axis=1)

non_val2_real_pred = doubled_combined.rename(columns={4: 'mac_true', 0:'mac_pred'})

merged_mac_real_pred = peaks.merge(non_val2_real_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
# m_mac_pred = merged_mac_real_pred[~merged_mac_real_pred[3].isin(nan['3_mac'])].reset_index(drop=True)
m_mac_pred = merged_mac_real_pred.rename(columns={3:'mac_peak'}).drop(4, axis=1)

####################### make the mouse doubled predicted and true df

mouse_pred = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model}/activations_mouse_TEST.csv', sep="\t", header=None)
mouse_real_neg=pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/test_splits/neg/mouse_liver_TEST_500bp.bed', sep="\t", header=None)
mouse_real_pos=pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/test_splits/log_pos/mouse_liver_TEST_500bp.bed', sep="\t", header=None)

real_neg_len = 2*len(mouse_real_neg)
real_pos_len = 2*len(mouse_real_pos)

mouse_pred_pos = mouse_pred.head(real_pos_len).reset_index(drop=True)

doubled_mouse_real_pos = pd.concat([mouse_real_pos, mouse_real_pos]).sort_index(kind='mergesort').reset_index(drop=True)


doubled_mouse_real_pos = doubled_mouse_real_pos.rename(columns={0:'mouse_chr'})
mouse_pos_real_pred = pd.concat([doubled_mouse_real_pos.drop(columns=5), mouse_pred_pos], axis=1)
mouse_pos_real_pred = mouse_pos_real_pred.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)


merged_mouse_pred = peaks.merge(mouse_pos_real_pred, how='left', left_on=14, right_on=3)

m_mouse_pred = merged_mouse_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

####################### make bigboss mouse + query predicted and real info

bigboss = pd.concat([m_mouse_pred, m_mac_pred], axis=1)

bigboss['true'] = bigboss['mouse_true']-bigboss['mac_true']
bigboss['pred'] = bigboss['mouse_pred']-bigboss['mac_pred']

mouse_av = (bigboss.loc[::2, 'mouse_pred'].values + bigboss.loc[1::2, 'mouse_pred'].values) / 2
mac_av = (bigboss.loc[::2, 'mac_pred'].values + bigboss.loc[1::2, 'mac_pred'].values) / 2

# Add the averages back to the DataFrame as a new column
bigboss.loc[::2, 'mouse_pred_avg'] = mouse_av  # Assign averages to even indices only
bigboss.loc[::2, 'mac_pred_avg'] = mac_av 
bigboss['avg_pred'] = bigboss['mouse_pred_avg']-bigboss['mac_pred_avg']


same_sign = np.sign(bigboss['true']) == np.sign(bigboss['pred'])
print(f'# of same sign for true difference and predicted difference: {same_sign.sum()} / {len(bigboss)} = {same_sign.sum() / len(bigboss)} %')

same_sign = np.sign(bigboss.loc[::2]['true']) == np.sign(bigboss.loc[::2]['avg_pred'])
print(f'# of same sign for true difference and averaged predicted difference: {same_sign.sum()} / {len(bigboss) / 2} = {same_sign.sum() / (len(bigboss) / 2)}%')

x = bigboss['true']
y = bigboss['pred']

print(pearson_spearman(x, y))

# of same sign for true difference and predicted difference: 466 / 798 = 0.5839598997493735 %
# of same sign for true difference and averaged predicted difference: 240 / 399.0 = 0.6015037593984962%
Pearson correlation coefficient: 0.212, p-value: 1.37e-09
Spearman correlation coefficient: 0.202, p-value: 8.24e-09
None


In [48]:
a = non_val2_real_pred

In [4]:
# 5 BEST LOG MODELS FOLDCHANGE
import pandas as pd
import scipy.stats

def correlate():
    rows = []
    # Lists for correlation calculations
    groups = ['Val2', 'Test2']
    dfs = [val2_foldchange_df, test2_foldchange_df]

    # Calculate correlations
    for group, df in zip(groups, dfs):
        x = df['true'].squeeze()
        y = df['pred'].squeeze()
        pearson, pp = scipy.stats.pearsonr(x, y)
        spearman, ps = scipy.stats.spearmanr(x, y)

        same_sign = np.sign(df['true']) == np.sign(df['pred'])
        num_ss = same_sign.sum()
        len_ss = len(df)
        perc_ss = num_ss / len_ss
        
        same_sign_avg = np.sign(df.loc[::2]['true']) == np.sign(df.loc[::2]['avg_pred'])
        num_ssa = same_sign_avg.sum()
        len_ssa = len(df) / 2
        perc_ssa = num_ssa / len_ssa
        
        rows.append({'Group': group, 'Metric': 'Same Sign Count', 'Value': num_ss})
        rows.append({'Group': group, 'Metric': 'Total Count', 'Value': len_ss})
        rows.append({'Group': group, 'Metric': 'Same Sign %', 'Value': perc_ss})
     
        rows.append({'Group': group, 'Metric': 'Pearson', 'Value': pearson})
        rows.append({'Group': group, 'Metric': 'Pearson P-Val', 'Value': pp})
        rows.append({'Group': group, 'Metric': 'Spearman', 'Value': spearman})
        rows.append({'Group': group, 'Metric': 'Spearman P-Val', 'Value': ps})
        
    return pd.DataFrame(rows)

# --- Main Script ---
all_results = []
species_list = ['macaque', 'rat', 'cow', 'pig']
model_list = ['bdbi7l3n', '7vsdq5k2', 'wnfdrgcc', '8i7h7nsh', 'ph4wrpxu']

for species in species_list:
    for model in model_list:
        model_dir = f'{model}_FINAL'

        nonMouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/{species}_liver_pos_ALL.bed', sep="\t", header=None)
        mouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/mouse_liver_pos_ALL.bed', sep="\t", header=None)
        one_to_one_peaks_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/oneToOnePeaks/{species}_mouse.bed', header=None, sep='\t')
        one_to_one_peaks_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/oneToOnePeaks/{species}_mouse.bed', header=None, sep='\t')

        peaks = one_to_one_peaks_VAL[[4, 14]]

        merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_non'))
        
        # Aligned orthologous peaks
        merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_NON', '_mouse'))

        #############################################################################
        # load MOUSE VAL DF

        pred_mouse_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_mouse_TRAIN_VAL.csv', header=None)
        
        mouse_train = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/logPos/mouse_liver_TRAINONLY.narrowPeak', sep="\t", header=None)
        mouse_val = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/logPos/mouse_liver_VAL.narrowPeak', sep="\t", header=None)
        
        mouse_train_len = 2*len(mouse_train)
        mouse_val_len = 2*len(mouse_val)

        doubled_mouse_val_df = pd.concat([mouse_val, mouse_val]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_mouse_val_df = doubled_mouse_val_df.rename(columns={0:'mouse_chr'})
    
        pred_mouse_val_df = pred_mouse_VAL.iloc[mouse_train_len:mouse_train_len + mouse_val_len].reset_index(drop=True)
        pred_mouse_val_df = pd.concat([doubled_mouse_val_df.drop(columns=5), pred_mouse_val_df], axis=1)
        pred_mouse_val_df = pred_mouse_val_df.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)
        
        merged_mouse_val_true_pred = peaks.merge(pred_mouse_val_df, how='left', left_on=14, right_on=3)
        mouse_val_true_pred = merged_mouse_val_true_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

        #############################################################################
        # load non-mouse VAL2 DF

        pred_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_VAL.csv', header=None)
        
        val1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/val_splits/val1/{species}_liver_VAL_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        val2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/log_val2/{species}_liver_VAL.narrowPeak', header=None, delim_whitespace=True).rename(columns={0: 'mac_chr'})
        
        val1_len = 2*len(val1_df)
        val2_len = 2*len(val2_df)
        
        doubled_val2_df = pd.concat([val2_df, val2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        pred_val2_df = pred_VAL.iloc[val1_len:val1_len + val2_len].reset_index(drop=True)
        doubled_val2_combined = pd.concat([doubled_val2_df, pred_val2_df], axis=1)

        non_val2_true_pred = doubled_val2_combined.rename(columns={4: 'non_true', 0:'non_pred'})
        merged_non_val2_true_pred = peaks.merge(non_val2_true_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
        non_val2_true_pred = merged_non_val2_true_pred.rename(columns={3:'non_peak'}).drop(4, axis=1)
        #############################################################################

        # make VAL2 merged df
        val2_foldchange_df = pd.concat([mouse_val_true_pred, non_val2_true_pred], axis=1)
        
        val2_foldchange_df['true'] = val2_foldchange_df['mouse_true']-val2_foldchange_df['non_true']
        val2_foldchange_df['pred'] = val2_foldchange_df['mouse_pred']-val2_foldchange_df['non_pred']
        
        mouse_av = (val2_foldchange_df.loc[::2, 'mouse_pred'].values + val2_foldchange_df.loc[1::2, 'mouse_pred'].values) / 2
        mac_av = (val2_foldchange_df.loc[::2, 'non_pred'].values + val2_foldchange_df.loc[1::2, 'non_pred'].values) / 2
        
        # Add the averages back to the DataFrame as a new column
        val2_foldchange_df.loc[::2, 'mouse_pred_avg'] = mouse_av
        val2_foldchange_df.loc[::2, 'non_pred_avg'] = mac_av 
        val2_foldchange_df['avg_pred'] = val2_foldchange_df['mouse_pred_avg']-val2_foldchange_df['non_pred_avg']

        #############################################################################
        # TIME FOR TEST2 FOLDCHANGE

        peaks = one_to_one_peaks_TEST[[4, 14]]

        merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_non'))
        
        # Aligned orthologous peaks
        merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_NON', '_mouse'))

        #############################################################################
        
        # load MOUSE TEST DF

        pred_mouse_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_mouse_TEST.csv', header=None)
        
        mouse_test = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/test_splits/log_pos/mouse_liver_TEST_500bp.bed', sep="\t", header=None)
        
        mouse_test_len = 2*len(mouse_test)

        doubled_mouse_test_df = pd.concat([mouse_test, mouse_test]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_mouse_test_df = doubled_mouse_test_df.rename(columns={0:'mouse_chr'})
    
        pred_mouse_test_df = pred_mouse_TEST.head(mouse_test_len).reset_index(drop=True)
        pred_mouse_test_df = pd.concat([doubled_mouse_test_df.drop(columns=5), pred_mouse_test_df], axis=1)
        pred_mouse_test_df = pred_mouse_test_df.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)
        
        merged_mouse_test_true_pred = peaks.merge(pred_mouse_test_df, how='left', left_on=14, right_on=3)
        mouse_test_true_pred = merged_mouse_test_true_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

        #############################################################################
        # load nonmouse TEST2 DF
        
        pred_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST.csv', header=None)
        
        test1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_LiuAll_test1/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        test2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_test2/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).rename(columns={0: 'mac_chr'})
        
        test1_len = 2*len(test1_df)
        test2_len = 2*len(test2_df)
        
        doubled_test2_df = pd.concat([test2_df, test2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        pred_test2_df = pred_TEST.iloc[test1_len:test1_len + test2_len].reset_index(drop=True)
        doubled_test2_combined = pd.concat([doubled_test2_df, pred_test2_df], axis=1)

        non_test2_true_pred = doubled_test2_combined.rename(columns={4: 'non_true', 0:'non_pred'})
        merged_non_test2_true_pred = peaks.merge(non_test2_true_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
        non_test2_true_pred = merged_non_test2_true_pred.rename(columns={3:'non_peak'}).drop(4, axis=1)

        #############################################################################

        # make TEST2 merged df
        test2_foldchange_df = pd.concat([mouse_test_true_pred, non_test2_true_pred], axis=1)
        
        test2_foldchange_df['true'] = test2_foldchange_df['mouse_true']-test2_foldchange_df['non_true']
        test2_foldchange_df['pred'] = test2_foldchange_df['mouse_pred']-test2_foldchange_df['non_pred']
        
        mouse_av = (test2_foldchange_df.loc[::2, 'mouse_pred'].values + test2_foldchange_df.loc[1::2, 'mouse_pred'].values) / 2
        non_av = (test2_foldchange_df.loc[::2, 'non_pred'].values + test2_foldchange_df.loc[1::2, 'non_pred'].values) / 2
        
        # Add the averages back to the DataFrame as a new column
        test2_foldchange_df.loc[::2, 'mouse_pred_avg'] = mouse_av
        test2_foldchange_df.loc[::2, 'non_pred_avg'] = non_av 
        test2_foldchange_df['avg_pred'] = test2_foldchange_df['mouse_pred_avg']-test2_foldchange_df['non_pred_avg']
        
        corr_df = correlate()
        corr_df['Species'] = species
        corr_df['model'] = model
        all_results.append(corr_df)
        


#############################################################################

summary_df = pd.concat(all_results)

custom_group_order = [ 'Val2', 'Test2' ]
custom_metric_order = ['Same Sign Count', 'Total Count', 'Same Sign %', 'Pearson', 'Pearson P-Val', 'Spearman', 'Spearman P-Val']

# # Convert 'group' to a categorical type with the specified order.
summary_df['Group'] = pd.Categorical(summary_df['Group'], categories=custom_group_order, ordered=True)
summary_df['Metric'] = pd.Categorical(summary_df['Metric'], categories=custom_metric_order, ordered=True)


pivot_df = summary_df.pivot_table(
    index=["Species", "Group", "Metric"],
    columns="model",
    values="Value"
).reset_index()

pivot_df = pivot_df.sort_values(by=["Species", "Group", "Metric"])

pivot_df = pivot_df[["Species", "Group", "Metric"] + model_list]
# pivot_df_reordered = pivot_df[model_list]

def format_value(metric, value):
    """Format values depending on whether it's a P-Val metric or not."""
    if "P-Val" in metric:
        return f"{value:.2e}"   # scientific notation, 3 sig figs
    else:
        return f"{value:.3g}"   # regular decimal, 3 sig figs

# Apply formatting to a copy just for display/export
pivot_df_display = pivot_df.copy()
for col in model_list:  # each model column
    pivot_df_display[col] = pivot_df_display.apply(
        lambda row: format_value(row["Metric"], row[col]),
        axis=1
    )

display(pivot_df_display)

output_filename = '/home/azstephe/liverRegression/regression_liver/data/figs/tables/log_model_foldchange_table_FINAL.tsv'
pivot_df_display.to_csv(output_filename, sep='\t')

# print(f'Results successfully saved to: {output_filename}')


model,Species,Group,Metric,bdbi7l3n,7vsdq5k2,wnfdrgcc,8i7h7nsh,ph4wrpxu
0,cow,Val2,Same Sign Count,200,184,184,183,181
1,cow,Val2,Total Count,330,330,330,330,330
2,cow,Val2,Same Sign %,0.606,0.558,0.558,0.555,0.548
3,cow,Val2,Pearson,0.259,0.286,0.273,0.25,0.273
4,cow,Val2,Pearson P-Val,1.90e-06,1.31e-07,4.67e-07,4.30e-06,4.87e-07
5,cow,Val2,Spearman,0.263,0.255,0.254,0.218,0.231
6,cow,Val2,Spearman P-Val,1.22e-06,2.80e-06,2.84e-06,6.45e-05,2.33e-05
7,cow,Test2,Same Sign Count,358,327,331,347,333
8,cow,Test2,Total Count,618,618,618,618,618
9,cow,Test2,Same Sign %,0.579,0.529,0.536,0.561,0.539


In [3]:
# 1 3 5 BEST LOG MODELS FOLDCHANGE
import pandas as pd
import scipy.stats

def correlate():
    rows = []
    # Lists for correlation calculations
    groups = ['Val2', 'Test2']
    dfs = [val2_foldchange_df, test2_foldchange_df]

    # Calculate correlations
    for group, df in zip(groups, dfs):
        x = df['true'].squeeze()
        y = df['pred'].squeeze()
        pearson, pp = scipy.stats.pearsonr(x, y)
        spearman, ps = scipy.stats.spearmanr(x, y)

        same_sign = np.sign(df['true']) == np.sign(df['pred'])
        num_ss = same_sign.sum()
        len_ss = len(df)
        perc_ss = num_ss / len_ss
        
        same_sign_avg = np.sign(df.loc[::2]['true']) == np.sign(df.loc[::2]['avg_pred'])
        num_ssa = same_sign_avg.sum()
        len_ssa = len(df) / 2
        perc_ssa = num_ssa / len_ssa
        
        rows.append({'Group': group, 'Metric': 'Same Sign Count', 'Value': num_ss})
        rows.append({'Group': group, 'Metric': 'Total Count', 'Value': len_ss})
        # print(f'total count: {len_ss}')
        rows.append({'Group': group, 'Metric': 'Same Sign %', 'Value': perc_ss})
     
        rows.append({'Group': group, 'Metric': 'Pearson', 'Value': pearson})
        rows.append({'Group': group, 'Metric': 'Pearson P-Val', 'Value': pp})
        rows.append({'Group': group, 'Metric': 'Spearman', 'Value': spearman})
        rows.append({'Group': group, 'Metric': 'Spearman P-Val', 'Value': ps})
        
    return pd.DataFrame(rows)

# --- Main Script ---
all_results = []
species_list = ['macaque', 'rat', 'cow', 'pig']
model_list = ['bdbi7l3n', 'kf8188qf', 'cq45eb2s']

for species in species_list:
    for model in model_list:
        model_dir = f'{model}_FINAL'

        nonMouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/{species}_liver_pos_ALL.bed', sep="\t", header=None)
        mouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/mouse_liver_pos_ALL.bed', sep="\t", header=None)
        one_to_one_peaks_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/oneToOnePeaks/{species}_mouse.bed', header=None, sep='\t')
        one_to_one_peaks_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/oneToOnePeaks/{species}_mouse.bed', header=None, sep='\t')

        peaks = one_to_one_peaks_VAL[[4, 14]]

        merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_non'))
        
        # Aligned orthologous peaks
        merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_NON', '_mouse'))

        #############################################################################
        # load MOUSE VAL DF

        pred_mouse_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_mouse_TRAIN_VAL.csv', header=None)
        
        mouse_train = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/logPos/mouse_liver_TRAINONLY.narrowPeak', sep="\t", header=None)
        mouse_val = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/logPos/mouse_liver_VAL.narrowPeak', sep="\t", header=None)
        
        mouse_train_len = 2*len(mouse_train)
        mouse_val_len = 2*len(mouse_val)

        doubled_mouse_val_df = pd.concat([mouse_val, mouse_val]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_mouse_val_df = doubled_mouse_val_df.rename(columns={0:'mouse_chr'})
    
        pred_mouse_val_df = pred_mouse_VAL.iloc[mouse_train_len:mouse_train_len + mouse_val_len].reset_index(drop=True)
        pred_mouse_val_df = pd.concat([doubled_mouse_val_df.drop(columns=5), pred_mouse_val_df], axis=1)
        pred_mouse_val_df = pred_mouse_val_df.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)
        
        merged_mouse_val_true_pred = peaks.merge(pred_mouse_val_df, how='left', left_on=14, right_on=3)
        mouse_val_true_pred = merged_mouse_val_true_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

        #############################################################################
        # load non-mouse VAL2 DF

        pred_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_VAL.csv', header=None)
        
        val1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/val_splits/val1/{species}_liver_VAL_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        val2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/log_val2/{species}_liver_VAL.narrowPeak', header=None, delim_whitespace=True).rename(columns={0: 'mac_chr'})
        
        val1_len = 2*len(val1_df)
        val2_len = 2*len(val2_df)
        
        doubled_val2_df = pd.concat([val2_df, val2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        pred_val2_df = pred_VAL.iloc[val1_len:val1_len + val2_len].reset_index(drop=True)
        doubled_val2_combined = pd.concat([doubled_val2_df, pred_val2_df], axis=1)

        non_val2_true_pred = doubled_val2_combined.rename(columns={4: 'non_true', 0:'non_pred'})
        merged_non_val2_true_pred = peaks.merge(non_val2_true_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
        non_val2_true_pred = merged_non_val2_true_pred.rename(columns={3:'non_peak'}).drop(4, axis=1)
        #############################################################################

        # make VAL2 merged df
        val2_foldchange_df = pd.concat([mouse_val_true_pred, non_val2_true_pred], axis=1)
        
        val2_foldchange_df['true'] = val2_foldchange_df['mouse_true']-val2_foldchange_df['non_true']
        val2_foldchange_df['pred'] = val2_foldchange_df['mouse_pred']-val2_foldchange_df['non_pred']
        
        mouse_av = (val2_foldchange_df.loc[::2, 'mouse_pred'].values + val2_foldchange_df.loc[1::2, 'mouse_pred'].values) / 2
        mac_av = (val2_foldchange_df.loc[::2, 'non_pred'].values + val2_foldchange_df.loc[1::2, 'non_pred'].values) / 2
        
        # Add the averages back to the DataFrame as a new column
        val2_foldchange_df.loc[::2, 'mouse_pred_avg'] = mouse_av
        val2_foldchange_df.loc[::2, 'non_pred_avg'] = mac_av 
        val2_foldchange_df['avg_pred'] = val2_foldchange_df['mouse_pred_avg']-val2_foldchange_df['non_pred_avg']

        #############################################################################
        # TIME FOR TEST2 FOLDCHANGE

        peaks = one_to_one_peaks_TEST[[4, 14]]

        merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_non'))
        
        # Aligned orthologous peaks
        merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_NON', '_mouse'))

        #############################################################################
        
        # load MOUSE TEST DF

        pred_mouse_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_mouse_TEST.csv', header=None)
        
        mouse_test = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/test_splits/log_pos/mouse_liver_TEST_500bp.bed', sep="\t", header=None)
        
        mouse_test_len = 2*len(mouse_test)

        doubled_mouse_test_df = pd.concat([mouse_test, mouse_test]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_mouse_test_df = doubled_mouse_test_df.rename(columns={0:'mouse_chr'})
    
        pred_mouse_test_df = pred_mouse_TEST.head(mouse_test_len).reset_index(drop=True)
        pred_mouse_test_df = pd.concat([doubled_mouse_test_df.drop(columns=5), pred_mouse_test_df], axis=1)
        pred_mouse_test_df = pred_mouse_test_df.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)
        
        merged_mouse_test_true_pred = peaks.merge(pred_mouse_test_df, how='left', left_on=14, right_on=3)
        mouse_test_true_pred = merged_mouse_test_true_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

        #############################################################################
        # load nonmouse TEST2 DF
        
        pred_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST.csv', header=None)
        
        test1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_LiuAll_test1/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        test2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_test2/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).rename(columns={0: 'mac_chr'})
        
        test1_len = 2*len(test1_df)
        test2_len = 2*len(test2_df)
        
        doubled_test2_df = pd.concat([test2_df, test2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        pred_test2_df = pred_TEST.iloc[test1_len:test1_len + test2_len].reset_index(drop=True)
        doubled_test2_combined = pd.concat([doubled_test2_df, pred_test2_df], axis=1)

        non_test2_true_pred = doubled_test2_combined.rename(columns={4: 'non_true', 0:'non_pred'})
        merged_non_test2_true_pred = peaks.merge(non_test2_true_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
        non_test2_true_pred = merged_non_test2_true_pred.rename(columns={3:'non_peak'}).drop(4, axis=1)

        #############################################################################

        # make TEST2 merged df
        test2_foldchange_df = pd.concat([mouse_test_true_pred, non_test2_true_pred], axis=1)
        
        test2_foldchange_df['true'] = test2_foldchange_df['mouse_true']-test2_foldchange_df['non_true']
        test2_foldchange_df['pred'] = test2_foldchange_df['mouse_pred']-test2_foldchange_df['non_pred']
        
        mouse_av = (test2_foldchange_df.loc[::2, 'mouse_pred'].values + test2_foldchange_df.loc[1::2, 'mouse_pred'].values) / 2
        non_av = (test2_foldchange_df.loc[::2, 'non_pred'].values + test2_foldchange_df.loc[1::2, 'non_pred'].values) / 2
        
        # Add the averages back to the DataFrame as a new column
        test2_foldchange_df.loc[::2, 'mouse_pred_avg'] = mouse_av
        test2_foldchange_df.loc[::2, 'non_pred_avg'] = non_av 
        test2_foldchange_df['avg_pred'] = test2_foldchange_df['mouse_pred_avg']-test2_foldchange_df['non_pred_avg']
        
        corr_df = correlate()
        corr_df['Species'] = species
        corr_df['model'] = model
        all_results.append(corr_df)
        


#############################################################################

summary_df = pd.concat(all_results)

custom_group_order = [ 'Val2', 'Test2' ]
custom_metric_order = ['Same Sign Count', 'Total Count', 'Same Sign %', 'Pearson', 'Pearson P-Val', 'Spearman', 'Spearman P-Val']

# # Convert 'group' to a categorical type with the specified order.
summary_df['Group'] = pd.Categorical(summary_df['Group'], categories=custom_group_order, ordered=True)
summary_df['Metric'] = pd.Categorical(summary_df['Metric'], categories=custom_metric_order, ordered=True)


pivot_df = summary_df.pivot_table(
    index=["Species", "Group", "Metric"],
    columns="model",
    values="Value"
).reset_index()

pivot_df = pivot_df.sort_values(by=["Species", "Group", "Metric"])

pivot_df = pivot_df[["Species", "Group", "Metric"] + model_list]
# pivot_df_reordered = pivot_df[model_list]

def format_value(metric, value):
    """Format values depending on whether it's a P-Val metric or not."""
    if "P-Val" in metric:
        return f"{value:.2e}"   # scientific notation, 3 sig figs
    else:
        return f"{value:.3g}"   # regular decimal, 3 sig figs

# Apply formatting to a copy just for display/export
pivot_df_display = pivot_df.copy()
for col in model_list:  # each model column
    pivot_df_display[col] = pivot_df_display.apply(
        lambda row: format_value(row["Metric"], row[col]),
        axis=1
    )

display(pivot_df_display)

output_filename = '/home/azstephe/liverRegression/regression_liver/data/figs/tables/135log_model_foldchange_table_FINAL.tsv'
pivot_df_display.to_csv(output_filename, sep='\t')

# print(f'Results successfully saved to: {output_filename}')


model,Species,Group,Metric,bdbi7l3n,kf8188qf,cq45eb2s
0,cow,Val2,Same Sign Count,200,211,193
1,cow,Val2,Total Count,330,330,330
2,cow,Val2,Same Sign %,0.606,0.639,0.585
3,cow,Val2,Pearson,0.259,0.355,0.274
4,cow,Val2,Pearson P-Val,1.90e-06,3.28e-11,4.16e-07
5,cow,Val2,Spearman,0.263,0.388,0.279
6,cow,Val2,Spearman P-Val,1.22e-06,2.54e-13,2.46e-07
7,cow,Test2,Same Sign Count,358,364,365
8,cow,Test2,Total Count,618,618,618
9,cow,Test2,Same Sign %,0.579,0.589,0.591


In [70]:
# 2KB MODEL FOLDCHANGE
import pandas as pd
import scipy.stats

def correlate():
    rows = []
    # Lists for correlation calculations
    groups = ['Test2']
    dfs = [test2_foldchange_df]

    # Calculate correlations
    for group, df in zip(groups, dfs):
        x = df['true'].squeeze()
        y = df['pred'].squeeze()
        pearson, pp = scipy.stats.pearsonr(x, y)
        spearman, ps = scipy.stats.spearmanr(x, y)

        same_sign = np.sign(df['true']) == np.sign(df['pred'])
        num_ss = same_sign.sum()
        len_ss = len(df)
        perc_ss = num_ss / len_ss
        
        same_sign_avg = np.sign(df.loc[::2]['true']) == np.sign(df.loc[::2]['avg_pred'])
        num_ssa = same_sign_avg.sum()
        len_ssa = len(df) / 2
        perc_ssa = num_ssa / len_ssa
        
        rows.append({'Group': group, 'Metric': 'Same Sign Count', 'Value': num_ss})
        rows.append({'Group': group, 'Metric': 'Total Count', 'Value': len_ss})
        # print(f'total count: {len_ss}')
        rows.append({'Group': group, 'Metric': 'Same Sign %', 'Value': perc_ss})
     
        rows.append({'Group': group, 'Metric': 'Pearson', 'Value': pearson})
        rows.append({'Group': group, 'Metric': 'Pearson P-Val', 'Value': pp})
        rows.append({'Group': group, 'Metric': 'Spearman', 'Value': spearman})
        rows.append({'Group': group, 'Metric': 'Spearman P-Val', 'Value': ps})
        
    return pd.DataFrame(rows)

# --- Main Script ---
all_results = []
species_list = ['macaque', 'rat', 'cow', 'pig']
model_list = ['im88hepv']

for species in species_list:
    for model in model_list:
        model_dir = f'{model}_FINAL'

        nonMouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/{species}_liver_pos_ALL.bed', sep="\t", header=None)
        mouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/mouse_liver_pos_ALL.bed', sep="\t", header=None)
        one_to_one_peaks_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/oneToOnePeaks/{species}_mouse.bed', header=None, sep='\t')

        peaks = one_to_one_peaks_TEST[[4, 14]]

        merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_non'))
        
        # Aligned orthologous peaks
        merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_NON', '_mouse'))

        #############################################################################
        
        # load MOUSE TEST DF

        pred_mouse_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_mouse_TEST.csv', header=None)
        
        mouse_test = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/test_splits_2kb/log_pos/mouse_liver_TEST_500bp.bed', sep="\t", header=None)
        
        mouse_test_len = 2*len(mouse_test)

        doubled_mouse_test_df = pd.concat([mouse_test, mouse_test]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_mouse_test_df = doubled_mouse_test_df.rename(columns={0:'mouse_chr'})
    
        pred_mouse_test_df = pred_mouse_TEST.head(mouse_test_len).reset_index(drop=True)
        pred_mouse_test_df = pd.concat([doubled_mouse_test_df, pred_mouse_test_df], axis=1)
        pred_mouse_test_df = pred_mouse_test_df.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)
        
        merged_mouse_test_true_pred = peaks.merge(pred_mouse_test_df, how='left', left_on=14, right_on=3)
        mouse_test_true_pred = merged_mouse_test_true_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

        #############################################################################
        # load nonmouse TEST2 DF
        
        pred_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST.csv', header=None)
        
        test1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits_2kb/log_LiuAll_test1/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        test2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits_2kb/log_test2/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).rename(columns={0: 'mac_chr'})
        
        test1_len = 2*len(test1_df)
        test2_len = 2*len(test2_df)
        
        doubled_test2_df = pd.concat([test2_df, test2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        pred_test2_df = pred_TEST.iloc[test1_len:test1_len + test2_len].reset_index(drop=True)
        doubled_test2_combined = pd.concat([doubled_test2_df, pred_test2_df], axis=1)

        non_test2_true_pred = doubled_test2_combined.rename(columns={4: 'non_true', 0:'non_pred'})
        merged_non_test2_true_pred = peaks.merge(non_test2_true_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
        non_test2_true_pred = merged_non_test2_true_pred.rename(columns={3:'non_peak'}).drop(4, axis=1)

        #############################################################################

        # make TEST2 merged df
        test2_foldchange_df = pd.concat([mouse_test_true_pred, non_test2_true_pred], axis=1)
        
        test2_foldchange_df['true'] = test2_foldchange_df['mouse_true']-test2_foldchange_df['non_true']
        test2_foldchange_df['pred'] = test2_foldchange_df['mouse_pred']-test2_foldchange_df['non_pred']
        
        mouse_av = (test2_foldchange_df.loc[::2, 'mouse_pred'].values + test2_foldchange_df.loc[1::2, 'mouse_pred'].values) / 2
        non_av = (test2_foldchange_df.loc[::2, 'non_pred'].values + test2_foldchange_df.loc[1::2, 'non_pred'].values) / 2
        
        # Add the averages back to the DataFrame as a new column
        test2_foldchange_df.loc[::2, 'mouse_pred_avg'] = mouse_av
        test2_foldchange_df.loc[::2, 'non_pred_avg'] = non_av 
        test2_foldchange_df['avg_pred'] = test2_foldchange_df['mouse_pred_avg']-test2_foldchange_df['non_pred_avg']
        
        corr_df = correlate()
        corr_df['Species'] = species
        corr_df['model'] = model
        all_results.append(corr_df)
        


#############################################################################

summary_df = pd.concat(all_results)

custom_metric_order = ['Same Sign Count', 'Total Count', 'Same Sign %', 'Pearson', 'Pearson P-Val', 'Spearman', 'Spearman P-Val']

summary_df['Metric'] = pd.Categorical(summary_df['Metric'], categories=custom_metric_order, ordered=True)

pivot_df = summary_df.pivot_table(
    index=["Species", "Group", "Metric"],
    columns="model",
    values="Value"
).reset_index()

pivot_df = pivot_df.sort_values(by=["Species", "Group", "Metric"])

pivot_df = pivot_df[["Species", "Group", "Metric"] + model_list]
# pivot_df_reordered = pivot_df[model_list]

def format_value(metric, value):
    """Format values depending on whether it's a P-Val metric or not."""
    if "P-Val" in metric:
        return f"{value:.2e}"   # scientific notation, 3 sig figs
    else:
        return f"{value:.3g}"   # regular decimal, 3 sig figs

# Apply formatting to a copy just for display/export
pivot_df_display = pivot_df.copy()
for col in model_list:  # each model column
    pivot_df_display[col] = pivot_df_display.apply(
        lambda row: format_value(row["Metric"], row[col]),
        axis=1
    )

pivot_df_2kb = pivot_df_display
display(pivot_df_display)

output_filename = '/home/azstephe/liverRegression/regression_liver/data/figs/tables/2kb_log_model_foldchange_table_FINAL.tsv'
pivot_df_2kb.to_csv(output_filename, sep='\t')

# print(f'Results successfully saved to: {output_filename}')


model,Species,Group,Metric,im88hepv
0,cow,Test2,Same Sign Count,327
1,cow,Test2,Total Count,618
2,cow,Test2,Same Sign %,0.529
3,cow,Test2,Pearson,0.199
4,cow,Test2,Pearson P-Val,5.93e-07
5,cow,Test2,Spearman,0.226
6,cow,Test2,Spearman P-Val,1.38e-08
7,macaque,Test2,Same Sign Count,461
8,macaque,Test2,Total Count,798
9,macaque,Test2,Same Sign %,0.578


In [63]:
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np

species = 'pig'

both_df = pd.read_csv(f"/home/azstephe/liverRegression/regression_liver/data/splits/{species}Mouse/{species}ToMouse_liver_{species}Enhancer_mouseEnhancer_test_wawb.narrowPeak", sep="\t", header=None)
mac_qn = pd.read_csv(f"/home/azstephe/liverRegression/regression_liver/data/ladder_qn/{species}_liver_pos_ALL.bed", sep="\t", header=None)
mouse_qn = pd.read_csv(f"/home/azstephe/liverRegression/regression_liver/data/ladder_qn/mouse_liver_pos_ALL.bed", sep="\t", header=None)

unique1_2 = both_df[~both_df.duplicated(subset=[1, 2], keep=False)] # entries with unique mac mapped to mouse start and end
all_unique = unique1_2[~unique1_2.duplicated(subset=[11,12], keep=False)]

#### FUNCTIONS

def pearson_spearman(x, y):
    pearson_corr, pearson_p_value = scipy.stats.pearsonr(x, y)
    print(f"Pearson correlation coefficient: {pearson_corr:.4f}, p-value: {pearson_p_value:.4g}")

    spearman_corr, spearman_p_value = scipy.stats.spearmanr(x, y)
    print(f"Spearman correlation coefficient: {spearman_corr:.4f}, p-value: {spearman_p_value:.4g}")
    
def collapse_if_identical(lst):
    return lst[0] if all(x == lst[0] for x in lst) else lst
    

# takes in list and returns peak with largest signal
def keep_largest_signal(lst, qn_df):
    if isinstance(lst, list):
        maxPeakName = ''
        maxPeakStrength = 0
        for peak in lst:
            matching_rows = qn_df[qn_df[3] == peak]
            if not matching_rows.empty:
                signal = matching_rows[4].iloc[0]
                if signal > maxPeakStrength:
                    maxPeakStrength = signal
                    maxPeakName = peak
        return maxPeakName
    else: 
        return lst

# def overlap(start1, end1, start2, end2):
#     return max(0, min(end1, end2)-max(start1, start2))

def get_biggest_overlap(lst, col, df):
    maxOverlap = 0
    maxPeakOverlap = ""
    for peak in lst:
        row = df[df[col] == peak].iloc[0] #iloc bc possible to return multiple rows (never will based on th
        overlap = max(0, min(row[2], row[12])-max(row[1], row[11]))
        if overlap > maxOverlap:
            maxOverlap = overlap
            maxPeakOverlap = peak
    return peak

#### COL1,2 DUPLICATES

duplicates3 = both_df[both_df.duplicated(subset=[1, 2], keep=False)] # entries with duplicated mac mapped to mouse start and end 
grouped_dups3 = duplicates3.groupby([1, 2])[3].apply(list).reset_index(name='col3')

# grouped dups: start | end | [peaks with these endpoints]
grouped_dups3['col3'] = grouped_dups3['col3'].apply(collapse_if_identical) 

grouped_dups3['col3'] = grouped_dups3['col3'].apply(lambda x: keep_largest_signal(x, mac_qn)) # redundant endpoints now map to 1 peak in col3
grouped_dups3 = grouped_dups3[grouped_dups3['col3'] != ''] # remove macaque peaks that aren't large enough

# df of duplicated col1,2 with strongest peak
keep_strongestcol3 = duplicates3.merge(grouped_dups3[['col3']], left_on=3, right_on='col3', how='inner').drop('col3', axis=1) # keeps the strongest signal in col3 for redundant endpoints

#### COL11,12 DUPLICATES

grouped_dups13 = keep_strongestcol3.groupby([11, 12])[13].apply(list).reset_index(name='col13')
grouped_dups13['col13'] = grouped_dups13['col13'].apply(collapse_if_identical)

grouped_dups13['col13'] = grouped_dups13['col13'].apply(lambda x: keep_largest_signal(x, mouse_qn))
grouped_dups13 = grouped_dups13[grouped_dups13['col13'] != '']

keep_strongestcol13 = keep_strongestcol3.merge(grouped_dups13[['col13']], left_on=13, right_on='col13', how='inner').drop('col13', axis=1)

unique_endpoints = keep_strongestcol13 #rows with unique endpoints from the duplicated endpoints set

####

# col3 peakname duplicates with different endpoints
still_dups_col3 = unique_endpoints[unique_endpoints.duplicated(subset=[3], keep=False)]

# col3peaks | [col13 peaks intersecting col3 peak]
grouped_dcol3 = still_dups_col3.groupby(3)[13].apply(list).reset_index(name='col13')

# get the col13 peak with most overlap of col3
grouped_dcol3['col13'] = grouped_dcol3['col13'].apply(lambda x: get_biggest_overlap(x, 13, still_dups_col3))

merged3 = still_dups_col3.merge(grouped_dcol3[[3, 'col13']], left_on=[3, 13], right_on=[3, 'col13'], how='left', indicator=True)

remove3 = merged3[merged3['_merge'] == 'left_only'].drop(columns=['_merge']) # col13 is what we want to remove

unique_endpoints_subset = unique_endpoints.iloc[:,:20]
remove3_subset = remove3.iloc[:,:20]

# all col3 entries unique
unique3 = unique_endpoints[~unique_endpoints_subset.apply(tuple, axis=1).isin(remove3_subset.apply(tuple, axis=1))] 

####

# col13 peakname duplicates with different endpoints
still_dups_col13 = unique3[unique3.duplicated(subset=[13], keep=False)]

# col13peaks | [col3 peaks intersecting col13 peak]
grouped_dcol13 = still_dups_col13.groupby(13)[3].apply(list).reset_index(name='col3')

# get the col3 peak with most overlap of col13
grouped_dcol13['col3'] = grouped_dcol13['col3'].apply(lambda x: get_biggest_overlap(x, 3, still_dups_col13))

merged13 = still_dups_col13.merge(grouped_dcol13[[13, 'col3']], left_on=[3, 13], right_on=['col3', 13], how='left', indicator=True)

remove13 = merged13[merged13['_merge'] == 'left_only'].drop(columns=['_merge'])

unique3_subset = unique3.iloc[:, :20]
remove13_subset = remove13.iloc[:, :20]

# Identify rows in u3 that are NOT in remove3
filtered_peaks_unique = unique3[~unique3_subset.apply(tuple, axis=1).isin(remove13_subset.apply(tuple, axis=1))]

full_unique = pd.concat([all_unique, filtered_peaks_unique])

# Sort by column '1'
one_to_one_peaks = full_unique.sort_values(by=1).reset_index(drop=True)

one_to_one_peaks.to_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/oneToOnePeaks_eqn/{species}_mouse_test.bed', header=None, sep='\t')

one_to_one_peaks

##############################



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,chr2,3229118,3229378,peak3020,-1,.,-1,-1,-1,151,chr2,3228532,3229401,peak11427,1000,.,16.724,152.951,149.789,727
1,chr2,4222111,4222486,peak3028,-1,.,-1,-1,-1,272,chr2,4222038,4222501,peak11441,868,.,6.236,32.050,29.533,356
2,chr1,4543950,4544442,peak13844,-1,.,-1,-1,-1,289,chr1,4543909,4544544,peak22,1000,.,10.428,103.923,100.955,398
3,chr2,5519359,5519700,peak3179,-1,.,-1,-1,-1,199,chr2,5519128,5519647,peak11454,983,.,5.291,23.969,21.543,364
4,chr2,6072172,6072362,peak3200,-1,.,-1,-1,-1,106,chr2,6072019,6072463,peak11460,1000,.,10.449,99.040,96.094,249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,chr1,194995147,194995697,peak19951,-1,.,-1,-1,-1,211,chr1,194995180,194995620,peak1965,1000,.,8.096,68.960,66.170,196
268,chr1,195016830,195017261,peak19954,-1,.,-1,-1,-1,137,chr1,195017180,195017466,peak1968,1000,.,5.099,40.066,37.475,169
269,chr1,195018208,195018410,peak19955,-1,.,-1,-1,-1,104,chr1,195017987,195018471,peak1969,1000,.,11.097,148.403,145.257,336
270,chr1,195033815,195034284,peak19958,-1,.,-1,-1,-1,315,chr1,195033987,195034343,peak1970,1000,.,12.148,108.731,105.741,201


In [69]:
# EQN MODEL FOLDCHANGE
import pandas as pd
import scipy.stats

def correlate():
    rows = []
    # Lists for correlation calculations
    groups = ['Test2']
    dfs = [test2_foldchange_df]

    # Calculate correlations
    for group, df in zip(groups, dfs):
        x = df['true'].squeeze()
        y = df['pred'].squeeze()
        pearson, pp = scipy.stats.pearsonr(x, y)
        spearman, ps = scipy.stats.spearmanr(x, y)

        same_sign = np.sign(df['true']) == np.sign(df['pred'])
        num_ss = same_sign.sum()
        len_ss = len(df)
        perc_ss = num_ss / len_ss
        
        same_sign_avg = np.sign(df.loc[::2]['true']) == np.sign(df.loc[::2]['avg_pred'])
        num_ssa = same_sign_avg.sum()
        len_ssa = len(df) / 2
        perc_ssa = num_ssa / len_ssa
        
        rows.append({'Group': group, 'Metric': 'Same Sign Count', 'Value': num_ss})
        rows.append({'Group': group, 'Metric': 'Total Count', 'Value': len_ss})
        # print(f'total count: {len_ss}')
        rows.append({'Group': group, 'Metric': 'Same Sign %', 'Value': perc_ss})
     
        rows.append({'Group': group, 'Metric': 'Pearson', 'Value': pearson})
        rows.append({'Group': group, 'Metric': 'Pearson P-Val', 'Value': pp})
        rows.append({'Group': group, 'Metric': 'Spearman', 'Value': spearman})
        rows.append({'Group': group, 'Metric': 'Spearman P-Val', 'Value': ps})
        
    return pd.DataFrame(rows)

# --- Main Script ---
all_results = []
species_list = ['macaque', 'rat', 'cow', 'pig']
model_list = ['mcf297qb', '7l12zan1']

for species in species_list:
    for model in model_list:
        model_dir = f'{model}_FINAL'

        nonMouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/{species}_liver_pos_ALL.bed', sep="\t", header=None)
        mouse_true = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/log/mouse_liver_pos_ALL.bed', sep="\t", header=None)
        one_to_one_peaks_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/oneToOnePeaks_eqn/{species}_mouse_test.bed', header=None, sep='\t')

        peaks = one_to_one_peaks_TEST[[4, 14]]

        merged_df = peaks.merge(nonMouse_true, how='left', left_on=4, right_on=3, suffixes=('', '_non'))
        
        # Aligned orthologous peaks
        merged_df = merged_df.merge(mouse_true, how='left', left_on=14, right_on=3, suffixes=('_NON', '_mouse'))

        #############################################################################
        
        # load MOUSE TEST DF

        pred_mouse_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_mouse_TEST.csv', header=None)
        
        mouse_test = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/test_splits_eqn/log_pos/mouse_liver_TEST_500bp.bed', sep="\t", header=None)
        
        mouse_test_len = 2*len(mouse_test)

        doubled_mouse_test_df = pd.concat([mouse_test, mouse_test]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_mouse_test_df = doubled_mouse_test_df.rename(columns={0:'mouse_chr'})
    
        pred_mouse_test_df = pred_mouse_TEST.head(mouse_test_len).reset_index(drop=True)
        pred_mouse_test_df = pd.concat([doubled_mouse_test_df, pred_mouse_test_df], axis=1)
        pred_mouse_test_df = pred_mouse_test_df.rename(columns={4:'mouse_true', 0:'mouse_pred'}).reset_index(drop=True)
        
        merged_mouse_test_true_pred = peaks.merge(pred_mouse_test_df, how='left', left_on=14, right_on=3)
        mouse_test_true_pred = merged_mouse_test_true_pred.rename(columns={3:'mouse_peak'}).drop([14], axis=1)

        #############################################################################
        # load nonmouse TEST2 DF
        
        pred_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST.csv', header=None)
        
        test1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_LiuAll_test1/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        test2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits_eqn/log_test2/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).rename(columns={0: 'mac_chr'})
        
        test1_len = 2*len(test1_df)
        test2_len = 2*len(test2_df)
        
        doubled_test2_df = pd.concat([test2_df, test2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        pred_test2_df = pred_TEST.iloc[test1_len:test1_len + test2_len].reset_index(drop=True)
        doubled_test2_combined = pd.concat([doubled_test2_df, pred_test2_df], axis=1)

        non_test2_true_pred = doubled_test2_combined.rename(columns={4: 'non_true', 0:'non_pred'})
        merged_non_test2_true_pred = peaks.merge(non_test2_true_pred, how='left', left_on=4, right_on=3).drop(14, axis=1)
        non_test2_true_pred = merged_non_test2_true_pred.rename(columns={3:'non_peak'}).drop(4, axis=1)

        #############################################################################

        # make TEST2 merged df
        test2_foldchange_df = pd.concat([mouse_test_true_pred, non_test2_true_pred], axis=1)
        
        test2_foldchange_df['true'] = test2_foldchange_df['mouse_true']-test2_foldchange_df['non_true']
        test2_foldchange_df['pred'] = test2_foldchange_df['mouse_pred']-test2_foldchange_df['non_pred']
        
        mouse_av = (test2_foldchange_df.loc[::2, 'mouse_pred'].values + test2_foldchange_df.loc[1::2, 'mouse_pred'].values) / 2
        non_av = (test2_foldchange_df.loc[::2, 'non_pred'].values + test2_foldchange_df.loc[1::2, 'non_pred'].values) / 2
        
        # Add the averages back to the DataFrame as a new column
        test2_foldchange_df.loc[::2, 'mouse_pred_avg'] = mouse_av
        test2_foldchange_df.loc[::2, 'non_pred_avg'] = non_av 
        test2_foldchange_df['avg_pred'] = test2_foldchange_df['mouse_pred_avg']-test2_foldchange_df['non_pred_avg']
        
        corr_df = correlate()
        corr_df['Species'] = species
        corr_df['model'] = model
        all_results.append(corr_df)
        


#############################################################################

summary_df = pd.concat(all_results)

custom_metric_order = ['Same Sign Count', 'Total Count', 'Same Sign %', 'Pearson', 'Pearson P-Val', 'Spearman', 'Spearman P-Val']

summary_df['Metric'] = pd.Categorical(summary_df['Metric'], categories=custom_metric_order, ordered=True)

pivot_df = summary_df.pivot_table(
    index=["Species", "Group", "Metric"],
    columns="model",
    values="Value"
).reset_index()

pivot_df = pivot_df.sort_values(by=["Species", "Group", "Metric"])

pivot_df = pivot_df[["Species", "Group", "Metric"] + model_list]
# pivot_df_reordered = pivot_df[model_list]

def format_value(metric, value):
    """Format values depending on whether it's a P-Val metric or not."""
    if "P-Val" in metric:
        return f"{value:.2e}"   # scientific notation, 3 sig figs
    else:
        return f"{value:.3g}"   # regular decimal, 3 sig figs

# Apply formatting to a copy just for display/export
pivot_df_display = pivot_df.copy()
for col in model_list:  # each model column
    pivot_df_display[col] = pivot_df_display.apply(
        lambda row: format_value(row["Metric"], row[col]),
        axis=1
    )

pivot_df_eqn = pivot_df_display
display(pivot_df_display)

output_filename = '/home/azstephe/liverRegression/regression_liver/data/figs/tables/eqn_log_model_foldchange_table_FINAL.tsv'
pivot_df_eqn.to_csv(output_filename, sep='\t')

# print(f'Results successfully saved to: {output_filename}')


model,Species,Group,Metric,mcf297qb,7l12zan1
0,cow,Test2,Same Sign Count,380,363
1,cow,Test2,Total Count,618,618
2,cow,Test2,Same Sign %,0.615,0.587
3,cow,Test2,Pearson,0.202,0.221
4,cow,Test2,Pearson P-Val,4.19e-07,2.70e-08
5,cow,Test2,Spearman,0.225,0.226
6,cow,Test2,Spearman P-Val,1.61e-08,1.29e-08
7,macaque,Test2,Same Sign Count,446,455
8,macaque,Test2,Total Count,798,798
9,macaque,Test2,Same Sign %,0.559,0.57
